In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd

In [ ]:
cd 'drive'/'MyDrive'/'Dacon'/'LDI'

/content/drive/MyDrive/Dacon/LDI


In [ ]:
train_err_data = pd.read_csv('./dataset/train_err_data.csv',thousands = ',')
train_quality_data = pd.read_csv('./dataset/train_quality_data.csv',thousands = ',')
train_problem_data = pd.read_csv('./dataset/train_problem_data.csv',thousands = ',')

In [ ]:
df_tqd = train_quality_data.drop_duplicates()
df_ted = train_err_data.drop_duplicates()
df_tpd = train_problem_data.drop_duplicates()

In [ ]:
df_ted.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [ ]:
df_ted['errtype'].value_counts()

23    2276011
31    1985037
22    1863495
15    1562517
4     1291986
16    1248142
32    1130247
40     730283
5      630280
26     530998
12     318847
11     305205
14     239212
28     215338
27     210131
33     178271
34     114833
41      99368
10      68832
7       51972
6       50433
17      45923
24      39064
42      34297
3       30497
2       26099
1       21513
39      15819
13      12123
35      10163
37       9336
36       9317
38       4539
18       2671
20       2080
25       1184
19        664
21        607
30        402
8         157
9         109
Name: errtype, dtype: int64

In [ ]:
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'connection timeout'),'errtype'] = '23_ct'
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'active'),'errtype'] = '23_as'
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'standby'),'errtype'] = '23_ac'
df_ted.loc[(df_ted['errtype'] == 31) & (df_ted['errcode'] == '0'),'errtype'] = '31_0'
df_ted.loc[(df_ted['errtype'] == 40) & (df_ted['errcode'] == '0'),'errtype'] = '40_0'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
encodedfwvlst = ['04.16.3553','04.22.1750','04.33.1261','05.15.2138','04.22.1778',  # Encoding ted by fwversion
                 '04.33.1185','03.11.1167']
lst_fwv = list(df_ted['fwver'])
for i in range(len(lst_fwv)):
    if lst_fwv[i] not in encodedfwvlst:
        lst_fwv[i] = 'nope'
df_ted['fwver'] = lst_fwv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
encodedfwvlst = ['04.16.3553','04.22.1750','04.33.1261','05.15.2138','04.22.1778' # Encoding tqd by fwversion
                 '04.33.1185','03.11.1167']
lst_fwv = list(df_tqd['fwver'])
for i in range(len(lst_fwv)):
    if lst_fwv[i] not in encodedfwvlst:
        lst_fwv[i] = 'nope'
df_tqd['fwver'] = lst_fwv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
df_ted['fwver'].value_counts()

04.16.3553    5145420
04.22.1750    2748374
04.33.1261    2375758
05.15.2138    1841272
04.22.1778    1243732
04.33.1185     911660
03.11.1167     845808
nope           255978
Name: fwver, dtype: int64

In [ ]:
for i in range(0,13):
    qstr = 'quality_'+str(i)
    b = df_tqd.loc[df_tqd[qstr] != -1,qstr] = 0

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [ ]:
df_ted = df_ted.drop('fwver',axis=1)
df_ted = df_ted.drop('model_nm',axis=1)
df_tqd = df_tqd.drop('fwver',axis=1)
df_ted = df_ted.drop('errcode',axis=1)

In [ ]:
df_encoded_ted = pd.get_dummies(df_ted,columns=['errtype'])

In [ ]:
df_encoded_ted

,user_id,time,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,errtype_23_ac,errtype_23_as,errtype_23_ct,errtype_31_0,errtype_40_0
0,10000,20201101025616,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10000,20201101030309,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10000,20201101030309,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10000,20201101050514,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10000,20201101050515,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16554658,24999,20201130163051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16554659,24999,20201130172625,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16554660,24999,20201130172625,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16554661,24999,20201130172631,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
lst_column = ['errtype_1', 'errtype_2', 'errtype_3', 'errtype_4', 'errtype_5', 'errtype_6', 'errtype_7', 'errtype_8', 'errtype_9', 'errtype_10', 'errtype_11', 'errtype_12', 'errtype_13', 'errtype_14', 'errtype_15', 'errtype_16', 'errtype_17', 'errtype_18', 'errtype_19', 'errtype_20', 'errtype_21', 'errtype_22', 'errtype_23', 'errtype_24', 'errtype_25', 'errtype_26', 'errtype_27', 'errtype_28', 'errtype_30', 'errtype_31', 'errtype_32', 'errtype_33', 'errtype_34', 'errtype_35', 'errtype_36', 'errtype_37', 'errtype_38', 'errtype_39', 'errtype_40', 'errtype_41', 'errtype_42', 'errtype_23_ac', 'errtype_23_as', 'errtype_23_ct', 'errtype_31_0', 'errtype_40_0', 'quality_0', 'quality_1', 'quality_2', 'quality_3', 'quality_4', 'quality_5', 'quality_6', 'quality_7', 'quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']# Make a output datafarme
df_x = pd.DataFrame(columns = lst_column)
df_x

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,errtype_23_ac,errtype_23_as,errtype_23_ct,errtype_31_0,errtype_40_0,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


In [ ]:
for i in range(len(df_tpd)):
    print(str(i),"/",str(len(df_tpd)),"finished.")
    time = df_tpd.loc[i].time
    user_id = df_tpd.loc[i].user_id
    df_check_time_ted = df_encoded_ted[(df_encoded_ted.user_id == user_id) & (df_encoded_ted.time <= time)]
    df_check_time_tqd = df_tqd[(df_tqd.user_id == user_id) & (df_tqd.time <= time)]
    df_check_time_ted = df_check_time_ted.drop('time',axis = 1)
    df_temp_ted = df_check_time_ted.drop('user_id',axis = 1)
    df_check_time_tqd = df_check_time_tqd.drop('time',axis = 1)
    df_temp_tqd = df_check_time_tqd.drop('user_id',axis = 1)
    df_res = pd.concat([np.sum(df_temp_ted), np.sum(df_temp_tqd)])
    df_x = df_x.append(pd.Series(df_res,index = df_x.columns),ignore_index=True)
    


0 / 5429 finished.
1 / 5429 finished.
2 / 5429 finished.
3 / 5429 finished.
4 / 5429 finished.
5 / 5429 finished.
6 / 5429 finished.
7 / 5429 finished.
8 / 5429 finished.
9 / 5429 finished.
10 / 5429 finished.
11 / 5429 finished.
12 / 5429 finished.
13 / 5429 finished.
14 / 5429 finished.
15 / 5429 finished.
16 / 5429 finished.
17 / 5429 finished.
18 / 5429 finished.
19 / 5429 finished.
20 / 5429 finished.
21 / 5429 finished.
22 / 5429 finished.
23 / 5429 finished.
24 / 5429 finished.
25 / 5429 finished.
26 / 5429 finished.
27 / 5429 finished.
28 / 5429 finished.
29 / 5429 finished.
30 / 5429 finished.
31 / 5429 finished.
32 / 5429 finished.
33 / 5429 finished.
34 / 5429 finished.
35 / 5429 finished.
36 / 5429 finished.
37 / 5429 finished.
38 / 5429 finished.
39 / 5429 finished.
40 / 5429 finished.
41 / 5429 finished.
42 / 5429 finished.
43 / 5429 finished.
44 / 5429 finished.
45 / 5429 finished.
46 / 5429 finished.
47 / 5429 finished.
48 / 5429 finished.
49 / 5429 finished.
50 / 5429 

In [ ]:
df_x

,user_id,model_nm,fwver,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,19224.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
1,23664.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.111111,-0.111111,-0.111111,0.0,0.0,0.888889,-0.111111,0.0,0.0,0.0,3.222222,-0.111111,0.0
2,23664.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.285714,-0.285714,-0.285714,0.0,0.0,0.428571,-0.285714,0.0,0.0,0.0,2.571429,-0.285714,0.0
3,15166.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.910508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.250000,-0.250000,-0.250000,0.0,0.0,0.500000,-0.250000,0.0,0.0,0.0,3.000000,-0.250000,0.0
4,12590.0,1.0,1.0,0.0,0.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6234,20167.0,0.0,3.0,0.0,0.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
6235,20167.0,0.0,4.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
6236,16270.0,2.0,5.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
6237,19114.0,3.0,9.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0


In [ ]:
import pickle
with open('list.txt', 'wb') as f:
    pickle.dump(df_x, f)